## 1. Preparing our dataset
<p><em>These recommendations are so on point! How does this playlist know me so well?</em></p>
<p><img src="https://s3.amazonaws.com/assets.datacamp.com/production/project_449/img/iphone_music.jpg" alt="Project Image Record" width="600px"></p>
<p>Over the past few years, streaming services with huge catalogs have become the primary means through which most people listen to their favorite music. But at the same time, the sheer amount of music on offer can mean users might be a bit overwhelmed when trying to look for newer music that suits their tastes.</p>
<p>For this reason, streaming services have looked into means of categorizing music to allow for personalized recommendations. One method involves direct analysis of the raw audio information in a given song, scoring the raw data on a variety of metrics. Today, we'll be examining data compiled by a research group known as The Echo Nest. Our goal is to look through this dataset and classify songs as being either 'Hip-Hop' or 'Rock' - all without listening to a single one ourselves. In doing so, we will learn how to clean our data, do some exploratory data visualization, and use feature reduction towards the goal of feeding our data through some simple machine learning algorithms, such as decision trees and logistic regression.</p>
<p>To begin with, let's load the metadata about our tracks alongside the track metrics compiled by The Echo Nest. A song is about more than its title, artist, and number of listens. We have another dataset that has musical features of each track such as <code>danceability</code> and <code>acousticness</code> on a scale from -1 to 1. These exist in two different files, which are in different formats - CSV and JSON. While CSV is a popular file format for denoting tabular data, JSON is another common file format in which databases often return the results of a given query.</p>
<p>Let's start by creating two pandas <code>DataFrames</code> out of these files that we can merge so we have features and labels (often also referred to as <code>X</code> and <code>y</code>) for the classification later on.</p>

In [28]:
## beautifull tables
from IPython.display import display
from IPython.display import Image
import pandas as pd

In [40]:
# Read in track metadata with genre labels
tracks = pd.read_csv("datasets/fma-rock-vs-hiphop.csv", sep=',' , encoding='utf-8')

In [16]:
# Read in track metrics with the features
echonest_metrics = pd.read_csv("datasets/echonest-metrics.json" , precise_float=True ) 

In [58]:
type(tracks)
type(echonest_metrics)

pandas.core.frame.DataFrame

In [42]:
echonest_metrics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Columns: 118161 entries, {"track_id":{"0":2 to 13128:0.2046522687}}
dtypes: object(118161)
memory usage: 0.0+ bytes


In [29]:
tracks

,track_id,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,...,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
0,135,256000,1,NaN,2008-11-26 01:43:26,2008-11-26 00:00:00,837,0,Rock,"[45, 58]",...,NaN,2484,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1832,NaN,0,NaN,[],Father's Day
1,136,256000,1,NaN,2008-11-26 01:43:35,2008-11-26 00:00:00,509,0,Rock,"[45, 58]",...,NaN,1948,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1498,NaN,0,NaN,[],Peel Back The Mountain Sky
2,151,192000,0,NaN,2008-11-26 01:44:55,NaN,192,0,Rock,[25],...,NaN,701,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,148,NaN,4,NaN,[],Untitled 04
3,152,192000,0,NaN,2008-11-26 01:44:58,NaN,193,0,Rock,[25],...,NaN,637,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,98,NaN,11,NaN,[],Untitled 11
4,153,256000,0,Arc and Sender,2008-11-26 01:45:00,2008-11-26 00:00:00,405,5,Rock,[26],...,NaN,354,en,Attribution-NonCommercial-NoDerivatives (aka M...,424,NaN,2,NaN,[],Hundred-Year Flood
5,154,256000,0,Arc and Sender,2008-11-26 01:45:04,2008-11-26 00:00:00,319,1,Rock,[26],...,NaN,242,en,Attribution-NonCommercial-NoDerivatives (aka M...,205,NaN,4,NaN,[],Squares And Circles
6,155,192000,0,NaN,2008-11-26 01:45:10,2008-11-26 00:00:00,756,1,Rock,[26],...,NaN,268,en,Attribution-NonCommercial-NoDerivatives (aka M...,197,NaN,0,NaN,[],Maps of the Stars Homes
7,169,192000,0,James Squeaky,2008-11-26 01:46:02,2006-01-01 00:00:00,144,1,Rock,[25],...,NaN,815,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,270,NaN,1,NaN,[],Boss of Goth
8,170,192000,0,NaN,2008-11-26 01:46:05,2006-10-07 00:00:00,181,0,Rock,[25],...,NaN,468,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,122,NaN,2,NaN,[],Industry Standard Massacre
9,171,320000,0,NaN,2008-11-26 01:46:07,2008-11-26 00:00:00,80,0,Rock,[25],...,<p>Unreleased</p>,582,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,242,NaN,2,NaN,[],Marching as Technitions


In [35]:
import numpy as np
tracks['track_id'].astype(np.int64)


0           135
1           136
2           151
3           152
4           153
5           154
6           155
7           169
8           170
9           171
10          172
11          173
12          174
13          175
14          176
15          177
16          178
17          179
18          181
19          182
20          183
21          184
22          226
23          228
24          247
25          249
26          252
27          255
28          256
29          257
          ...  
17704    152545
17705    153333
17706    153334
17707    153335
17708    153336
17709    153337
17710    154292
17711    154293
17712    154294
17713    154295
17714    154296
17715    154297
17716    154298
17717    154299
17718    154300
17719    154301
17720    154302
17721    154303
17722    154304
17723    154305
17724    154306
17725    154307
17726    154308
17727    154309
17728    154919
17729    155063
17730    155064
17731    155065
17732    155066
17733    155247
Name: track_id, Length: 

In [87]:
tracks.shape

(17734, 21)

In [88]:
echonest_metrics.shape


(0, 118161)

In [101]:
# Merge the relevant columns of tracks and echonest_metrics  
echonest_metrics = pd.DataFrame(echonest_metrics)
tracks = pd.DataFrame(tracks)
#tracks.convert_objects(convert_numeric=True)

#pd.merge(echonest_metrics, tracks ,how='left', on="track_id"  ) 
 
#print(pd.merge(echonest_metrics, tracks ,  how='left', left_on='track_id', right_on='track_id' ))

#echonest_metrics.join(tracks.set_index('track_id'), on='track_id')

#echo_tracks

AttributeError: 'DataFrame' object has no attribute 'splitlines'

In [96]:
df1 = pd.DataFrame(columns=['a','b','c'], data=[[1,2,3],[4,5,6],[7,8,9],[10,11,12]])
df1_c_index = df1.set_index('c')
df2 = pd.DataFrame(columns=['d','e'], data=[[3,14],[6,15],[13,16]])

type(df1)

print ('df1', '\n', df1, '\n')
print ('df1_c_index', '\n', df1_c_index, '\n')
print ('df2', '\n', df2, '\n')

print ("pd.merge(df1, df2, how='left', left_on='c', right_on='d')", '\n')
print (pd.merge(df1, df2, how='left', left_on='c', right_on='d'), '\n')

print ("pd.merge(df1_c_index, df2, how='left', left_index=True, right_on='d')", '\n')
print (pd.merge(df1_c_index, df2, how='left', left_index=True, right_on='d'), '\n')

df2.rename(columns={'d':'c'}, inplace=True)

print ('df1', '\n', df1, '\n')
print ('df1_c_index', '\n', df1_c_index, '\n')
print ('df2', '\n', df2, '\n')

print ("pd.merge(df1, df2, how='left', left_on='c', right_on='c')", '\n')
print (pd.merge(df1, df2, how='left', left_on='c', right_on='c'), '\n')

print ("pd.merge(df1_c_index, df2, how='left', left_index=True, right_on='c')", '\n')
print (pd.merge(df1_c_index, df2, how='left', left_index=True, right_on='c'), '\n')

df1 
     a   b   c
0   1   2   3
1   4   5   6
2   7   8   9
3  10  11  12 

df1_c_index 
      a   b
c         
3    1   2
6    4   5
9    7   8
12  10  11 

df2 
     d   e
0   3  14
1   6  15
2  13  16 

pd.merge(df1, df2, how='left', left_on='c', right_on='d') 

    a   b   c    d     e
0   1   2   3  3.0  14.0
1   4   5   6  6.0  15.0
2   7   8   9  NaN   NaN
3  10  11  12  NaN   NaN 

pd.merge(df1_c_index, df2, how='left', left_index=True, right_on='d') 

    a   b   d     e
0   1   2   3  14.0
1   4   5   6  15.0
2   7   8   9   NaN
2  10  11  12   NaN 

df1 
     a   b   c
0   1   2   3
1   4   5   6
2   7   8   9
3  10  11  12 

df1_c_index 
      a   b
c         
3    1   2
6    4   5
9    7   8
12  10  11 

df2 
     c   e
0   3  14
1   6  15
2  13  16 

pd.merge(df1, df2, how='left', left_on='c', right_on='c') 

    a   b   c     e
0   1   2   3  14.0
1   4   5   6  15.0
2   7   8   9   NaN
3  10  11  12   NaN 

pd.merge(df1_c_index, df2, how='left', left_index=True, right

In [ ]:

# Inspect the resultant dataframe
echo_tracks.info()

## 2. Pairwise relationships between continuous variables
<p>We typically want to avoid using variables that have strong correlations with each other -- hence avoiding feature redundancy -- for a few reasons:</p>
<ul>
<li>To keep the model simple and improve interpretability (with many features, we run the risk of overfitting).</li>
<li>When our datasets are very large, using fewer features can drastically speed up our computation time.</li>
</ul>
<p>To get a sense of whether there are any strongly correlated features in our data, we will use built-in functions in the <code>pandas</code> package.</p>

In [0]:
# Create a correlation matrix
corr_metrics = ...
corr_metrics.style.background_gradient()

## 3. Normalizing the feature data
<p>As mentioned earlier, it can be particularly useful to simplify our models and use as few features as necessary to achieve the best result. Since we didn't find any particular strong correlations between our features, we can instead use a common approach to reduce the number of features called <strong>principal component analysis (PCA)</strong>. </p>
<p>It is possible that the variance between genres can be explained by just a few features in the dataset. PCA rotates the data along the axis of highest variance, thus allowing us to determine the relative contribution of each feature of our data towards the variance between classes. </p>
<p>However, since PCA uses the absolute variance of a feature to rotate the data, a feature with a broader range of values will overpower and bias the algorithm relative to the other features. To avoid this, we must first normalize our data. There are a few methods to do this, but a common way is through <em>standardization</em>, such that all features have a mean = 0 and standard deviation = 1 (the resultant is a z-score).</p>

In [0]:
# Define our features 
features = ...

# Define our labels
labels = ...

# Import the StandardScaler
...

# Scale the features and set the values to a new variable
scaler = ...
scaled_train_features = ...

## 4. Principal Component Analysis on our scaled data
<p>Now that we have preprocessed our data, we are ready to use PCA to determine by how much we can reduce the dimensionality of our data. We can use <strong>scree-plots</strong> and <strong>cumulative explained ratio plots</strong> to find the number of components to use in further analyses.</p>
<p>Scree-plots display the number of components against the variance explained by each component, sorted in descending order of variance. Scree-plots help us get a better sense of which components explain a sufficient amount of variance in our data. When using scree plots, an 'elbow' (a steep drop from one data point to the next) in the plot is typically used to decide on an appropriate cutoff.</p>

In [0]:
# This is just to make plots appear in the notebook
%matplotlib inline

# Import our plotting module, and PCA class
#... YOUR CODE ...

# Get our explained variance ratios from PCA using all features
pca = ...
...
exp_variance = ...

# plot the explained variance using a barplot
fig, ax = plt.subplots()
ax.bar(..., ...)
ax.set_xlabel('Principal Component #')

## 5. Further visualization of PCA
<p>Unfortunately, there does not appear to be a clear elbow in this scree plot, which means it is not straightforward to find the number of intrinsic dimensions using this method. </p>
<p>But all is not lost! Instead, we can also look at the <strong>cumulative explained variance plot</strong> to determine how many features are required to explain, say, about 90% of the variance (cutoffs are somewhat arbitrary here, and usually decided upon by 'rules of thumb'). Once we determine the appropriate number of components, we can perform PCA with that many components, ideally reducing the dimensionality of our data.</p>

In [0]:
# Import numpy
...

# Calculate the cumulative explained variance
cum_exp_variance = ...

# Plot the cumulative explained variance and draw a dashed line at 0.90.
fig, ax = plt.subplots()
...
ax.axhline(y=0.9, linestyle='--')
n_components = ...

# Perform PCA with the chosen number of components and project data onto components
pca = PCA(n_components, random_state=10)
pca.fit(scaled_train_features)
pca_projection = ...

## 6. Train a decision tree to classify genre
<p>Now we can use the lower dimensional PCA projection of the data to classify songs into genres. To do that, we first need to split our dataset into 'train' and 'test' subsets, where the 'train' subset will be used to train our model while the 'test' dataset allows for model performance validation.</p>
<p>Here, we will be using a simple algorithm known as a decision tree. Decision trees are rule-based classifiers that take in features and follow a 'tree structure' of binary decisions to ultimately classify a data point into one of two or more categories. In addition to being easy to both use and interpret, decision trees allow us to visualize the 'logic flowchart' that the model generates from the training data.</p>
<p>Here is an example of a decision tree that demonstrates the process by which an input image (in this case, of a shape) might be classified based on the number of sides it has and whether it is rotated.</p>
<p><img src="https://s3.amazonaws.com/assets.datacamp.com/production/project_449/img/simple_decision_tree.png" alt="Decision Tree Flow Chart Example" width="350px"></p>

In [0]:
# Import train_test_split function and Decision tree classifier
# ... YOUR CODE ...

# Split our data
train_features, test_features, train_labels, test_labels = ...

# Train our decision tree
tree = ...
...

# Predict the labels for the test data
pred_labels_tree = ...

## 7. Compare our decision tree to a logistic regression
<p>Although our tree's performance is decent, it's a bad idea to immediately assume that it's therefore the perfect tool for this job -- there's always the possibility of other models that will perform even better! It's always a worthwhile idea to at least test a few other algorithms and find the one that's best for our data.</p>
<p>Sometimes simplest is best, and so we will start by applying <strong>logistic regression</strong>. Logistic regression makes use of what's called the logistic function to calculate the odds that a given data point belongs to a given class. Once we have both models, we can compare them on a few performance metrics, such as false positive and false negative rate (or how many points are inaccurately classified). </p>

In [0]:
# Import LogisticRegression
from sklearn.linear_model import LogisticRegression

# Train our logistic regression and predict labels for the test set
logreg = ...
...
pred_labels_logit = ...

# Create the classification report for both models
from sklearn.metrics import classification_report
class_rep_tree = ...
class_rep_log = ...

print("Decision Tree: \n", class_rep_tree)
print("Logistic Regression: \n", class_rep_log)

## 8. Balance our data for greater performance
<p>Both our models do similarly well, boasting an average precision of 87% each. However, looking at our classification report, we can see that rock songs are fairly well classified, but hip-hop songs are disproportionately misclassified as rock songs. </p>
<p>Why might this be the case? Well, just by looking at the number of data points we have for each class, we see that we have far more data points for the rock classification than for hip-hop, potentially skewing our model's ability to distinguish between classes. This also tells us that most of our model's accuracy is driven by its ability to classify just rock songs, which is less than ideal.</p>
<p>To account for this, we can weight the value of a correct classification in each class inversely to the occurrence of data points for each class. Since a correct classification for "Rock" is not more important than a correct classification for "Hip-Hop" (and vice versa), we only need to account for differences in <em>sample size</em> of our data points when weighting our classes here, and not relative importance of each class. </p>

In [0]:
# Subset only the hip-hop tracks, and then only the rock tracks
hop_only = ...
rock_only = ...

# sample the rocks songs to be the same number as there are hip-hop songs
rock_only = ...

# concatenate the dataframes rock_only and hop_only
rock_hop_bal = ...

# The features, labels, and pca projection are created for the balanced dataframe
features = rock_hop_bal.drop(['genre_top', 'track_id'], axis=1) 
labels = rock_hop_bal['genre_top']
pca_projection = pca.fit_transform(scaler.fit_transform(features))

# Redefine the train and test set with the pca_projection from the balanced data
train_features, test_features, train_labels, test_labels = train_test_split(..., random_state=10)

## 9. Does balancing our dataset improve model bias?
<p>We've now balanced our dataset, but in doing so, we've removed a lot of data points that might have been crucial to training our models. Let's test to see if balancing our data improves model bias towards the "Rock" classification while retaining overall classification performance. </p>
<p>Note that we have already reduced the size of our dataset and will go forward without applying any dimensionality reduction. In practice, we would consider dimensionality reduction more rigorously when dealing with vastly large datasets and when computation times become prohibitively large.</p>

In [0]:
# Train our decision tree on the balanced data
tree = ...
...
pred_labels_tree = ...

# Train our logistic regression on the balanced data
logreg = ...
...
pred_labels_logit = ...

# Compare the models
print("Decision Tree: \n", classification_report(...))
print("Logistic Regression: \n", classification_report(...))

## 10. Using cross-validation to evaluate our models
<p>Success! Balancing our data has removed bias towards the more prevalent class. To get a good sense of how well our models are actually performing, we can apply what's called <strong>cross-validation</strong> (CV). This step allows us to compare models in a more rigorous fashion.</p>
<p>Since the way our data is split into train and test sets can impact model performance, CV attempts to split the data multiple ways and test the model on each of the splits. Although there are many different CV methods, all with their own advantages and disadvantages, we will use what's known as <strong>K-fold</strong> CV here. K-fold first splits the data into K different, equally sized subsets. Then, it iteratively uses each subset as a test set while using the remainder of the data as train sets. Finally, we can then aggregate the results from each fold for a final model performance score.</p>

In [0]:
from sklearn.model_selection import KFold, cross_val_score

# Set up our K-fold cross-validation
kf = ...

tree = DecisionTreeClassifier(random_state=10)
logreg = LogisticRegression(random_state=10)

# Train our models using KFold cv
tree_score = ...
logit_score = ...

# Print the mean of each array of scores
print("Decision Tree:", ..., "Logistic Regression:", ...)